In [ ]:
import scipy.io as sio
import pandas as pd
import mne
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

## Detil file

### Dataset fif

In [ ]:

# Define the path to your .fif file
file_path = r"E:\Vscode\Tugas Anime\Dashboard\fif_directory\tmpi80tq398.fif"

# Load the epochs
epochs = mne.read_epochs(file_path, preload=True)

# Print the number of epochs
print("Number of epochs:", len(epochs))

# Optionally, print some additional information about the epochs
print(epochs)


### Dataset depression real

In [ ]:
file_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\alpha_band_C3-R_depression\MDD-1-epo_EEG_C3-R_Depression_alpha-band.edf"

# raw = mne.io.read_raw_edf(file_path, preload=True)
raw = mne.read_epochs(file_path, preload=True)

print(f"File info: {raw.info}")
print(f"Sampling rate: {raw.info['sfreq']} Hz")
print(f"Number of channels: {len(raw.info['ch_names'])}")
print(f"Channel names: {raw.info['ch_names']}")
print(f"Data shape: {raw.get_data().shape}")

## Konversi file dari mat ke fif
### Dataset healthy

In [ ]:
# Dataset Healthy
# Define the directory containing the .mat files
mat_directory = r'E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\file_mat_V2_healthy'
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\fif_file_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

mat_files = glob(os.path.join(mat_directory, '*.mat'))

# Function to extract the key for EEG data from the .mat file
def get_eeg_key(mat):
    for key in mat.keys():
        if key.startswith('a') and key.endswith('mat'):
            return key
    return None

# channel names
channels_to_plot = {
    'EEG039': 'FT7',
    'EEG045': 'T7',
    'EEG050': 'TP7',
    'EEG101': 'TP8',
    'EEG108': 'T8',
    'EEG115': 'FT8',
    'EEG024': 'F3',
    'EEG124': 'F4'
}

# Iterate over each .mat file in the directory
for mat_file_path in mat_files:
    # Load the .mat file
    mat = sio.loadmat(mat_file_path)
    
    # Extract the key for the EEG data
    eeg_key = get_eeg_key(mat)
    if eeg_key is None:
        print(f"EEG data key not found in {mat_file_path}. Skipping file.")
        continue
    
    # Load the EEG data, sampling rate, and define channel names
    data = mat[eeg_key]
    sampling_rate = mat['samplingRate'][0, 0]
    channel_names = ['EEG%03d' % (i + 1) for i in range(data.shape[0])]
    

    # Create MNE info object
    info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types='eeg')
    
    # Transpose data if necessary
    if data.shape[0] != len(channel_names):
        data = data.T
    
    # Create RawArray
    raw = mne.io.RawArray(data, info)

    # Rename Channels
    for original_name, new_name in channels_to_plot.items():
        if original_name in channel_names:
            mne.rename_channels(raw.info, {original_name: new_name})

    # Extract indices of the channels to plot
    indices_to_plot = [raw.ch_names.index(new_name) for new_name in channels_to_plot.values() if new_name in raw.ch_names]

    # plotting
    plot_path_raw = os.path.join(plot_directory, os.path.splitext(os.path.basename(mat_file_path))[0] + '_selected_channels.png')
    fig = raw.plot(order=indices_to_plot, title="Selected Channels", duration=6, scalings='auto', show=False)
    fig.subplots_adjust(top=0.85)
    fig.savefig(plot_path_raw)
    fig.clf()

    print(f'Saved plot of selected channels to {plot_path_raw}')
    
    # Define the output .fif filename based on the .mat filename
    fif_filename = os.path.splitext(os.path.basename(mat_file_path))[0] + '.fif'
    output_path = os.path.join(fif_directory, fif_filename)
    raw.save(output_path, overwrite=True)
    
    print(f'Converted {mat_file_path} to {output_path}')

#### Visualisasi per channel

In [ ]:
# Define the directory containing the .fif files
fif_directory = r'E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Healthy\Primary\format_fif_V2'
# Define the main output directory for plots
visualization_directory = r'E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Healthy\Primary\Visualization'

fif_files = glob(os.path.join(fif_directory, '*.fif'))

# Channel names to plot
channels_to_plot = {
    'EEG039': 'FT7',
    'EEG045': 'T7',
    'EEG050': 'TP7',
    'EEG101': 'TP8',
    'EEG108': 'T8',
    'EEG115': 'FT8',
    'EEG024': 'F3',
    'EEG124': 'F4'
}

# Create a directory for each channel
for channel in channels_to_plot.values():
    os.makedirs(os.path.join(visualization_directory, channel), exist_ok=True)

# Iterate over each .fif file in the directory
for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.io.read_raw_fif(fif_file_path, preload=True)
    
    # Plotting each selected channel individually
    for original_name, new_name in channels_to_plot.items():
        if new_name in raw.ch_names:
            channel_dir = os.path.join(visualization_directory, new_name)
            plot_path = os.path.join(channel_dir, os.path.splitext(os.path.basename(fif_file_path))[0] + f'_{new_name}.png')
            fig = raw.plot_psd(tmax=np.inf, picks=[new_name], show=False)
            fig.savefig(plot_path)
            plt.close(fig)
            print(f'Saved PSD plot for {new_name} to {plot_path}')

### Dataset Depresi

In [ ]:
# Dataset Depression
# Define the directory containing the .mat files
mat_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\file_mat_V2_depression"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\fif_directory_depression"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

mat_files = glob(os.path.join(mat_directory, '*.mat'))

# Function to extract the key for EEG data from the .mat file
def get_eeg_key(mat):
    for key in mat.keys():
        if key.startswith('a') and key.endswith('mat'):
            return key
    return None

# channel names
channels_to_plot = {
    'EEG039': 'FT7',
    'EEG045': 'T7',
    'EEG050': 'TP7',
    'EEG101': 'TP8',
    'EEG108': 'T8',
    'EEG115': 'FT8',
    'EEG024': 'F3',
    'EEG124': 'F4'
}

# Iterate over each .mat file in the directory
for mat_file_path in mat_files:
    # Load the .mat file
    mat = sio.loadmat(mat_file_path)
    
    # Extract the key for the EEG data
    eeg_key = get_eeg_key(mat)
    if eeg_key is None:
        print(f"EEG data key not found in {mat_file_path}. Skipping file.")
        continue
    
    # Load the EEG data, sampling rate, and define channel names
    data = mat[eeg_key]
    sampling_rate = mat['samplingRate'][0, 0]
    channel_names = ['EEG%03d' % (i + 1) for i in range(data.shape[0])]
    

    # Create MNE info object
    info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types='eeg')
    
    # Transpose data if necessary
    if data.shape[0] != len(channel_names):
        data = data.T
    
    # Create RawArray
    raw = mne.io.RawArray(data, info)

    # Rename Channels
    for original_name, new_name in channels_to_plot.items():
        if original_name in channel_names:
            mne.rename_channels(raw.info, {original_name: new_name})

    # Extract indices of the channels to plot
    indices_to_plot = [raw.ch_names.index(new_name) for new_name in channels_to_plot.values() if new_name in raw.ch_names]

    # plotting
    plot_path_raw = os.path.join(plot_directory, os.path.splitext(os.path.basename(mat_file_path))[0] + '_selected_channels.png')
    fig = raw.plot(order=indices_to_plot, title="Selected Channels", duration=6, scalings='auto', show=False)
    fig.subplots_adjust(top=0.85)
    fig.savefig(plot_path_raw)
    fig.clf()

    print(f'Saved plot of selected channels to {plot_path_raw}')
    
    # Define the output .fif filename based on the .mat filename
    fif_filename = os.path.splitext(os.path.basename(mat_file_path))[0] + '.fif'
    output_path = os.path.join(fif_directory, fif_filename)
    raw.save(output_path, overwrite=True)
    
    print(f'Converted {mat_file_path} to {output_path}')

#### Visualisasi per channel

In [ ]:
# Define the directory containing the .fif files
fif_directory = r'E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Depression\Primary\format_fif_V2'
# Define the main output directory for plots
visualization_directory = r'E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Depression\Primary\Visualization'

fif_files = glob(os.path.join(fif_directory, '*.fif'))

# Channel names to plot
channels_to_plot = {
    'EEG039': 'FT7',
    'EEG045': 'T7',
    'EEG050': 'TP7',
    'EEG101': 'TP8',
    'EEG108': 'T8',
    'EEG115': 'FT8',
    'EEG024': 'F3',
    'EEG124': 'F4'
}

# Create a directory for each channel
for channel in channels_to_plot.values():
    os.makedirs(os.path.join(visualization_directory, channel), exist_ok=True)

# Iterate over each .fif file in the directory
for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.io.read_raw_fif(fif_file_path, preload=True)
    
    # Plotting each selected channel individually
    for original_name, new_name in channels_to_plot.items():
        if new_name in raw.ch_names:
            channel_dir = os.path.join(visualization_directory, new_name)
            plot_path = os.path.join(channel_dir, os.path.splitext(os.path.basename(fif_file_path))[0] + f'_{new_name}.png')
            fig = raw.plot_psd(tmax=np.inf, picks=[new_name], show=False)
            fig.savefig(plot_path)
            plt.close(fig)
            print(f'Saved PSD plot for {new_name} to {plot_path}')

## Segmentasi sinyal jadi epoch per 6 detik dengan channel yang udah dipilih
### Dataset healthy

In [ ]:
# Define the directories
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\fif_file_healthy"
epoch_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
plot_directory = os.path.join(epoch_directory, 'plots')

# Ensure the directories exist
if not os.path.exists(epoch_directory):
    os.makedirs(epoch_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

# Load .fif files
fif_files = glob(os.path.join(fif_directory, '*.fif'))

# Define selected channels
selected_channels = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']

for fif_file_path in fif_files:
    raw = mne.io.read_raw_fif(fif_file_path, preload=True)
    
    # # Display and inspect the raw data
    # print(raw.info)
    # raw.plot(duration=5, n_channels=len(selected_channels), scalings='auto')
    
    # # Confirm to proceed
    # user_input = input("Proceed with processing this file? (y/n): ")
    # if user_input.lower() != 'y':
    #     continue

    # Create epochs
    events = mne.make_fixed_length_events(raw, start=0, duration=6)
    print(events)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=5.999, baseline=None, preload=True, picks=selected_channels)
    
    # print(f"Number of epochs created: {len(epochs)}")
    
    # Save epochs (original)
    epoch_filename = os.path.splitext(os.path.basename(fif_file_path))[0] + '-epo.fif'
    epochs.save(os.path.join(epoch_directory, epoch_filename), overwrite=True)

    # # Plot and save plot for each channel individually
    # for channel in selected_channels:
    #     # Define the subfolder name with prefix 'epoch6s_'
    #     subfolder_name = f'epoch6s_{channel}'
    #     # Ensure the directory for this channel exists
    #     channel_plot_directory = os.path.join(plot_directory, channel)
    #     if not os.path.exists(channel_plot_directory):
    #         os.makedirs(channel_plot_directory)
    
    # Plot and save epochs (original)
    fig = epochs.plot(picks=selected_channels, n_epochs=6, n_channels=len(selected_channels), scalings='auto', show=False)
    plot_path = os.path.join(plot_directory, os.path.splitext(os.path.basename(fif_file_path))[0] + '_selected_epochs.png')
    fig.savefig(plot_path)
    fig.clf()  # Clear the figure

        # fig = epochs.plot(picks=channel, n_epochs=6, scalings='auto', show=False)
        # plot_filename = os.path.splitext(os.path.basename(fif_file_path))[0] + f'_{channel}.png'
        # fig_path = os.path.join(channel_plot_directory, plot_filename)
        # fig.savefig(fig_path)
        # fig.clf()  # Clear the figure to save memory
        # print(f'Saved plot for {channel} in {fif_file_path}')

    # print(f'Saved epochs and plot for {fif_file_path}')


### Dataset depression

In [ ]:
# Define the directories
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\fif_directory_depression"
epoch_directory = r'E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Depression\Primary\6s epochs'
plot_directory = os.path.join(epoch_directory, 'plots')

# Ensure the directories exist
if not os.path.exists(epoch_directory):
    os.makedirs(epoch_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

# Load .fif files
fif_files = glob(os.path.join(fif_directory, '*.fif'))

# Define selected channels
selected_channels = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']

for fif_file_path in fif_files:
    raw = mne.io.read_raw_fif(fif_file_path, preload=True)
    
    # # Display and inspect the raw data
    # print(raw.info)
    # raw.plot(duration=5, n_channels=len(selected_channels), scalings='auto')
    
    # # Confirm to proceed
    # user_input = input("Proceed with processing this file? (y/n): ")
    # if user_input.lower() != 'y':
    #     continue

    # Create epochs
    events = mne.make_fixed_length_events(raw, start=0, duration=6)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=5.999, baseline=None, preload=True, picks=selected_channels)
    
    # Save epochs
    epoch_filename = os.path.splitext(os.path.basename(fif_file_path))[0] + '-epo.fif'
    epochs.save(os.path.join(epoch_directory, epoch_filename), overwrite=True)

    # Plot and save epochs
    fig = epochs.plot(picks=selected_channels, n_epochs=6, n_channels=len(selected_channels), scalings='auto', show=False)
    plot_path = os.path.join(plot_directory, os.path.splitext(os.path.basename(fif_file_path))[0] + '_selected_epochs.png')
    fig.savefig(plot_path)
    fig.clf()  # Clear the figure

    print(f'Saved epochs and plot for {fif_file_path}')


## Dataset real

### Depression


In [ ]:
# Define the directories
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\File_asli"
epoch_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\6s_epochs_depression"
plot_directory = os.path.join(epoch_directory, 'plots')

# Ensure the directories exist
if not os.path.exists(epoch_directory):
    os.makedirs(epoch_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

# Load .fif files
edf_files = glob(os.path.join(edf_directory, '*.edf'))

# Define selected channels
selected_channels = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']

for edf_file_path in edf_files:
    try:
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        
        # # Display and inspect the raw data
        # print(raw.info)
        # raw.plot(duration=5, n_channels=len(selected_channels), scalings='auto')
        
        # # Confirm to proceed
        # user_input = input("Proceed with processing this file? (y/n): ")
        # if user_input.lower() != 'y':
        #     continue

        # Create epochs
        events = mne.make_fixed_length_events(raw, start=0, duration=6)
        epochs = mne.Epochs(raw, events, tmin=0, tmax=5.999, baseline=None, preload=True, picks=selected_channels)
        
        # Save epochs
        epoch_filename = os.path.splitext(os.path.basename(edf_file_path))[0] + '-epo.edf'
        epochs.save(os.path.join(epoch_directory, epoch_filename), overwrite=True)

        # Plot and save epochs
        fig = epochs.plot(picks=selected_channels, n_epochs=6, n_channels=len(selected_channels), scalings='auto', show=False)
        plot_path = os.path.join(plot_directory, os.path.splitext(os.path.basename(edf_file_path))[0] + '_selected_epochs.png')
        fig.savefig(plot_path)
        fig.clf()  # Clear the figure

        print(f'Saved epochs and plot for {edf_file_path}')
    except Exception as e:
        print(f"Error processing {edf_file_path}: {e}")

### Healthy


In [ ]:
# Define the directories
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\File_asli"
epoch_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
plot_directory = os.path.join(epoch_directory, 'plots')

# Ensure the directories exist
if not os.path.exists(epoch_directory):
    os.makedirs(epoch_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

# Load .fif files
edf_files = glob(os.path.join(edf_directory, '*.edf'))

# Define selected channels
selected_channels = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']

for edf_file_path in edf_files:
    try:    
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        
        # # Display and inspect the raw data
        # print(raw.info)
        # raw.plot(duration=5, n_channels=len(selected_channels), scalings='auto')
        
        # # Confirm to proceed
        # user_input = input("Proceed with processing this file? (y/n): ")
        # if user_input.lower() != 'y':
        #     continue

        # Create epochs
        events = mne.make_fixed_length_events(raw, start=0, duration=6)
        epochs = mne.Epochs(raw, events, tmin=0, tmax=5.999, baseline=None, preload=True, picks=selected_channels)
        
        # Save epochs
        epoch_filename = os.path.splitext(os.path.basename(edf_file_path))[0] + '-epo.edf'
        epochs.save(os.path.join(epoch_directory, epoch_filename), overwrite=True)

        # Plot and save epochs
        fig = epochs.plot(picks=selected_channels, n_epochs=6, n_channels=len(selected_channels), scalings='auto', show=False)
        plot_path = os.path.join(plot_directory, os.path.splitext(os.path.basename(edf_file_path))[0] + '_selected_epochs.png')
        fig.savefig(plot_path)
        fig.clf()  # Clear the figure

        print(f'Saved epochs and plot for {edf_file_path}')
    except Exception as e:
        print(f"Error processing {edf_file_path}: {e}")


## Pasang filter
### Alpha

In [ ]:
# Define the directory containing the source files
fif_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\alpha_band_TP8_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

fif_files = glob(os.path.join(fif_source, '*.fif'))

for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.read_epochs(fif_file_path, preload=True)

    # Select channel 'EEG101' and filter for the alpha band (8-12 Hz)
    raw.pick_channels(['TP8']).filter(8, 12)

    # Saving the filtered signal
    filtered_filename = os.path.basename(fif_file_path).replace('.fif', '_TP8_Healthy_alpha-band.fif')
    filtered_output_path = os.path.join(fif_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.fif', '_psd.png'))
    raw.plot_psd(fmin=8, fmax=12, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### Alpha (dataset real)

In [ ]:
# Define the directory containing the source files
edf_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\alpha_band_T4-R_healthy"
plot_directory = os.path.join(edf_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(edf_directory):
    os.makedirs(edf_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

edf_files = glob(os.path.join(edf_source, '*.edf'))

for edf_file_path in edf_files:
    # Load the .fif file
    raw = mne.read_epochs(edf_file_path, preload=True)

    # Select channel 'EEG101' and filter for the alpha band (8-12 Hz)
    raw.pick_channels(['EEG T4-R']).filter(8, 12)
    # Saving the filtered signal
    filtered_filename = os.path.basename(edf_file_path).replace('.edf', '_EEG_T4-R_Healthy_alpha-band.edf')
    filtered_output_path = os.path.join(edf_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.edf', '_psd.png'))
    raw.plot_psd(fmin=8, fmax=12, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### Delta

In [ ]:
# Define the directory containing the source files
fif_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\delta_band_TP8_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

fif_files = glob(os.path.join(fif_source, '*.fif'))

for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.read_epochs(fif_file_path, preload=True)

    # Select channel and filter for the delta band (0.1-3 Hz)
    raw.pick_channels(['TP8']).filter(0.1, 3)

    # Saving the filtered signal
    filtered_filename = os.path.basename(fif_file_path).replace('.fif', '_TP8_Healthy_delta-band.fif')
    filtered_output_path = os.path.join(fif_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.fif', '_psd.png'))
    raw.plot_psd(fmin=0.1, fmax=3, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### delta (dataset real)

In [ ]:
# Define the directory containing the source files
edf_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
# Define the output directory for the .edf files
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\delta_band_T4-R_healthy"
plot_directory = os.path.join(edf_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(edf_directory):
    os.makedirs(edf_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

edf_files = glob(os.path.join(edf_source, '*.edf'))

for edf_file_path in edf_files:
    # Load the .edf file
    raw = mne.read_epochs(edf_file_path, preload=True)

    # Select channel and filter for the delta band (0.1-3 Hz)
    raw.pick_channels(['EEG T4-R']).filter(0.1, 3)

    # Saving the filtered signal
    filtered_filename = os.path.basename(edf_file_path).replace('.edf', '_EEG_T4-R_Healthy_delta-band.edf')
    filtered_output_path = os.path.join(edf_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.edf', '_psd.png'))
    raw.plot_psd(fmin=0.1, fmax=3, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### Theta

In [ ]:
# Define the directory containing the source files
fif_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\theta_band_TP8_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

fif_files = glob(os.path.join(fif_source, '*.fif'))

for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.read_epochs(fif_file_path, preload=True)

    # Select channel and filter for the theta band (4-7 Hz)
    raw.pick_channels(['TP8']).filter(4, 7)

    # Saving the filtered signal
    filtered_filename = os.path.basename(fif_file_path).replace('.fif', '_TP8_Healthy_theta-band.fif')
    filtered_output_path = os.path.join(fif_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.fif', '_psd.png'))
    raw.plot_psd(fmin=4, fmax=7, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### theta (dataset real)

In [ ]:
# Define the directory containing the source files
edf_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
# Define the output directory for the .edf files
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\theta_band_T4-R_healthy"
plot_directory = os.path.join(edf_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(edf_directory):
    os.makedirs(edf_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

edf_files = glob(os.path.join(edf_source, '*.edf'))

for edf_file_path in edf_files:
    # Load the .edf file
    raw = mne.read_epochs(edf_file_path, preload=True)

    # Select channel and filter for the theta band (4-7 Hz)
    raw.pick_channels(['EEG T4-R']).filter(4, 7)

    # Saving the filtered signal
    filtered_filename = os.path.basename(edf_file_path).replace('.edf', '_EEG_T4-R_Healthy_theta-band.edf')
    filtered_output_path = os.path.join(edf_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.edf', '_psd.png'))
    raw.plot_psd(fmin=4, fmax=7, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### Beta

In [ ]:
# Define the directory containing the source files
fif_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\beta_band_TP8_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

fif_files = glob(os.path.join(fif_source, '*.fif'))

for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.read_epochs(fif_file_path, preload=True)

    # Select channel and filter for the beta band (12-30 Hz)
    raw.pick_channels(['TP8']).filter(12, 30)

    # Saving the filtered signal
    filtered_filename = os.path.basename(fif_file_path).replace('.fif', '_TP8_Healthy_beta-band.fif')
    filtered_output_path = os.path.join(fif_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.fif', '_psd.png'))
    raw.plot_psd(fmin=12, fmax=30, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### beta (dataset real)

In [ ]:
# Define the directory containing the source files
edf_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
# Define the output directory for the .edf files
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\beta_band_T4-R_healthy"
plot_directory = os.path.join(edf_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(edf_directory):
    os.makedirs(edf_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

edf_files = glob(os.path.join(edf_source, '*.edf'))

for edf_file_path in edf_files:
    # Load the .edf file
    raw = mne.read_epochs(edf_file_path, preload=True)

    # Select channel and filter for the beta band (12-30 Hz)
    raw.pick_channels(['EEG T4-R']).filter(12, 30)

    # Saving the filtered signal
    filtered_filename = os.path.basename(edf_file_path).replace('.edf', '_EEG_T4-R_Depression_beta-band.edf')
    filtered_output_path = os.path.join(edf_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.edf', '_psd.png'))
    raw.plot_psd(fmin=12, fmax=30, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### gamma

In [ ]:
# Define the directory containing the source files
fif_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
# Define the output directory for the .fif files
fif_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\gamma_band_TP8_healthy"
plot_directory = os.path.join(fif_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(fif_directory):
    os.makedirs(fif_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

fif_files = glob(os.path.join(fif_source, '*.fif'))

for fif_file_path in fif_files:
    # Load the .fif file
    raw = mne.read_epochs(fif_file_path, preload=True)

    # Select channel and filter for the gamma band (30-45 Hz)
    raw.pick_channels(['TP8']).filter(30, 45)

    # Saving the filtered signal
    filtered_filename = os.path.basename(fif_file_path).replace('.fif', '_TP8_Healthy_gamma-band.fif')
    filtered_output_path = os.path.join(fif_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.fif', '_psd.png'))
    raw.plot_psd(fmin=30, fmax=45, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### gamma (dataset real)

In [ ]:
# Define the directory containing the source files
edf_source = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
# Define the output directory for the .edf files
edf_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\gamma_band_T4-R_healthy"
plot_directory = os.path.join(edf_directory, 'plots')

# Ensure the output directory exists
if not os.path.exists(edf_directory):
    os.makedirs(edf_directory)
if not os.path.exists(plot_directory):
    os.makedirs(plot_directory)

edf_files = glob(os.path.join(edf_source, '*.edf'))

for edf_file_path in edf_files:
    # Load the .edf file
    raw = mne.read_epochs(edf_file_path, preload=True)

    # Select channel and filter for the gamma band (30-45 Hz)
    raw.pick_channels(['EEG T4-R']).filter(30, 45)

    # Saving the filtered signal
    filtered_filename = os.path.basename(edf_file_path).replace('.edf', '_EEG_T4-R_Healthy_gamma-band.edf')
    filtered_output_path = os.path.join(edf_directory, filtered_filename)
    raw.save(filtered_output_path, overwrite=True)
    print(f'Filtered data saved to {filtered_output_path}')

    # Plotting the power spectral density of the filtered signal
    plot_path = os.path.join(plot_directory, filtered_filename.replace('.edf', '_psd.png'))
    raw.plot_psd(fmin=30, fmax=45, show=False).savefig(plot_path)
    print(f'Power spectral density plot saved to {plot_path}')

### Kombinasi fif buat jadi satu sinyal

In [ ]:
import os
from glob import glob
import mne
import matplotlib.pyplot as plt

# Path to your directory containing the .fif files
directory_path = r"E:\Vscode\Tugas Anime\Workspace\Dataset\EEG_128channels_resting_lanzhou_2015\Depression\Primary\6s_epochs"

# List all .fif files
fif_files = glob(os.path.join(directory_path, '*.fif'))

# Load and concatenate the epochs from each file
all_epochs = []
for file in fif_files:
    epochs = mne.read_epochs(file, preload=True)
    all_epochs.append(epochs)

# Concatenate all epochs into one object
combined_epochs = mne.concatenate_epochs(all_epochs)

# Plot the original PSD (Second Image)
fig_original = combined_epochs.plot_psd(fmin=0, fmax=100, average=False, show=False)
plt.show()

# Create a new figure for the highlighted PSD plot (Third Image)
fig_highlighted, ax = plt.subplots(figsize=(12, 6))  # Adjust the figure size as needed

# Highlight specific epochs
highlighted_epochs = [1, 2]  # indices of epochs to highlight
# colors = ['red', 'blue']  # colors for highlighting
cmap = plt.cm.get_cmap('viridis', 50)
colors = cmap(np.linspace(0, 1, len(highlighted_epochs)))

for i, epoch_idx in enumerate(highlighted_epochs):
    psd = combined_epochs[epoch_idx].compute_psd(fmin=0, fmax=100)
    freqs = psd.freqs
    psd_data = psd.get_data().mean(axis=1).squeeze()  # average across channels, then remove extra dimensions
    ax.plot(freqs, 10 * np.log10(psd_data), color=colors[i], label=f'Epoch {epoch_idx + 1}')

# Add legend and labels
ax.legend()
ax.set_title('Highlighted Epochs PSD')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density (dB)')

# Show the modified plot
plt.show()
